The purpose of this project is to generate evidence using Evidence Model.

---

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [2]:
content = []
with open('/content/gdrive/MyDrive/EvidenceQuery/debatepedia/test_content', "r") as d:
  for doc in d:
    content.append(doc[4:-7])

query = []
with open('/content/gdrive/MyDrive/EvidenceQuery/debatepedia/test_query', "r") as d:
  for doc in d:
    query.append(doc[4:-7])

summary = []
with open('/content/gdrive/MyDrive/EvidenceQuery/debatepedia/test_summary', "r") as d:
  for doc in d:
    summary.append(doc[4:-7])


In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
!pip install transformers
!pip install sentencepiece==0.1.91

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/MyDrive/EvidenceQuery/model').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def generateEvidence (text_document):
  input_ids = tokenizer.encode(text_document, padding=True, truncation = True, return_tensors='pt').to(device)
  output = model.generate(input_ids, max_length=12, min_length=12, no_repeat_ngram_size= 4)
  evidence = tokenizer.decode(output[0], skip_special_tokens=True)
  return evidence

#print(summary[200])
#print(generateEvidence(content[200]))

In [10]:
evidence = []

for i in range(len(content)):
   evidence.append(generateEvidence(content[i]))

In [21]:
import json

debatepedia_data = []

for i in range(len(content)):
  debatepedia_data.append({'src':content[i],'qry':query[i],'tgt':summary[i],'evd':evidence[i]})

with open('/content/gdrive/MyDrive/EvidenceQuery/debatepediaEvidence.json', 'w') as fp:
  json.dump(debatepedia_data, fp)